In [143]:
# https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

#
# This is a sample Notebook to demonstrate how to read "MNIST Dataset"
#
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)    

In [144]:
# # https://www.kaggle.com/code/hojjatk/read-mnist-dataset/notebook

# #
# # Verify Reading Dataset via MnistDataloader class
# #
# %matplotlib inline
# import random
# import matplotlib.pyplot as plt
# import pickle

# #
# # Set file paths based on added MNIST Datasets
# #
# input_path = 'dataset/'
# training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
# training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
# test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
# test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# #
# # Helper function to show a list of images with their relating titles
# #
# def show_images(images, title_texts):
#     cols = 5
#     rows = int(len(images)/cols) + 1
#     plt.figure(figsize=(30,20))
#     index = 1    
#     for x in zip(images, title_texts):        
#         image = x[0]        
#         title_text = x[1]
#         plt.subplot(rows, cols, index)        
#         plt.imshow(image, cmap=plt.cm.gray)
#         if (title_text != ''):
#             plt.title(title_text, fontsize = 15);        
#         index += 1

# #
# # Load MINST dataset
# #
# mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
# (data_train, label_train), (data_test, label_test) = mnist_dataloader.load_data()

# with open("dataset/pickled/data_train.pickle", "wb") as outfile:
#     pickle.dump(data_train, outfile)
# with open("dataset/pickled/label_train.pickle", "wb") as outfile:
#     pickle.dump(label_train, outfile)
# with open("dataset/pickled/data_test.pickle", "wb") as outfile:
#     pickle.dump(data_test, outfile)
# with open("dataset/pickled/label_test.pickle", "wb") as outfile:
#     pickle.dump(label_test, outfile)

# first_10_data_train = []
# first_10_label_train = []
# for i in range(0, 10):
#     first_10_data_train.append(data_train[i])
#     first_10_label_train.append(label_train[i])

# with open("dataset/pickled/first_10_data_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_data_train, outfile)
# with open("dataset/pickled/first_10_label_train.pickle", "wb") as outfile:
#     pickle.dump(first_10_label_train, outfile)

# #
# # Show some random training and test images 
# #
# images_2_show = []
# titles_2_show = []
# for i in range(0, 10):
#    images_2_show.append(first_10_data_train[i])
#    titles_2_show.append('training image [' + str(i) + '] = ' + str(first_10_label_train[i]))    

# #for i in range(0, 5):
# #   r = random.randint(1, 10000)
# #   images_2_show.append(data_test[r])        
# #   titles_2_show.append('test image [' + str(r) + '] = ' + str(label_test[r]))    

# show_images(images_2_show, titles_2_show)


In [145]:
import random
import numpy as np

class Neuron:
    def __init__(self, numberOfIncoming):
        self.activation = random.uniform(0, 1)
        self.bias = random.uniform(0, 1)
        self.incomingWeights = np.array([random.uniform(0, 1) for i in range(numberOfIncoming)])
        self.error = 0

    def cout(self):
        print("Activation: ", self.activation)
        print("Bias: ", self.bias)
        print("Incoming Weights: ", self.incomingWeights)
        print("Error: ", self.error)


In [146]:
class Layer:
    def __init__(self, prevLayerLen, currentLayerLen):
        self.layer = np.array([Neuron(prevLayerLen) for i in range(currentLayerLen)])
        self.zVector = np.zeros(shape = (currentLayerLen, 1))
        self.size = currentLayerLen

    def getActivationVector(self):
        return np.array([neur.activation for neur in self.layer])
    
    def coutNeurons(self):
        print("Neurons:")
        for neur in self.layer:
            neur.cout()
        print()

    def cout(self):
        self.coutNeurons()

        print("Z Vector:")
        print(self.zVector)
        print()

In [147]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidDeriv(x):
    return np.exp(-x) / (1 + np.exp(-x))

class Network:

    def __init__(self, start, first, second, end):
        startLayer = Layer(0, start)
        firstLayer = Layer(start, first)
        secondLayer = Layer(first, second)
        endLayer = Layer(second, end)

        self.Layers = np.array([startLayer, firstLayer, secondLayer, endLayer])

    def calculateZVecor(self, layerIdx):
        # weight matrix
        outLayer = self.Layers[layerIdx]
        inLayer = self.Layers[layerIdx - 1]
        weightMatrix = np.empty((outLayer.size, inLayer.size))
        for col in range (outLayer.size):
            for row in range (outLayer.layer[0].incomingWeights.size):
                weightMatrix[row][col] = outLayer.layer[col].incomingWeights[row]

        # input activation
        inActivation = np.array([neur.activation for neur in inLayer.layer])
        
        # output biases
        outBias = np.array([neur.bias for neur in outLayer.layer])

        outLayer.zVector = np.dot(weightMatrix, inActivation)
        outLayer.zVector = outLayer.zVector + outBias

        return outLayer.zVector

    def forwardPropagationStep(self, layerIdx):
        outLayer = self.Layers[layerIdx]
        inLayer = self.Layers[layerIdx - 1]
        outputMat = self.calculateZVecor(layerIdx)

        for i in range(outLayer.size):
            outLayer.layer[i].activation = sigmoid(outputMat[i])

    # Assumes that data is between 0 and 255 value
    def setStartLayerActivations(self, dataset):
        if len(dataset) * len(dataset[0]) != self.Layers[0].size:
            print("There is a mismatch between the size of the input data and the start layer!")
            print("Size of dataset is: " + str(len(dataset) * len(dataset[0])))
            print("Size of first layer is: " + str(self.Layers[0].size))

        print("1) Set the activations of the first layer")

        layerIdx = 0
        for row in range(0, len(dataset)):
            for col in range(0, len(dataset[0])):
                self.Layers[0].layer[layerIdx].activation = dataset[row][col] / 255
                layerIdx += 1

    def fullForwardPropagation(self, target):
        self.forwardPropagationStep(1)
        self.forwardPropagationStep(2)
        self.forwardPropagationStep(3)
        print("Target is: " + str(target))
        print("Cost is: " + str(self.cost(target)))
    
    def cost(self, target):
        sum = 0
        for i in range(self.Layers[self.Layers.size - 1].size):
            if (i == target):
                sum += pow(self.Layers[self.Layers.size - 1].layer[i].activation - 1.0, 2)
            else:
                sum += pow(self.Layers[self.Layers.size - 1].layer[i].activation, 2)
        return sum / 2 * self.Layers[self.Layers.size - 1].size
    
    # The cost function is hard coded
    def calculateErrorInLastLayerForTarget(self, target):
        CGradient = np.zeros(shape = (self.Layers[self.Layers.size - 1].size, 1))
        for i in range(0, self.Layers[self.Layers.size - 1].size):
            CGradient[i] = self.Layers[self.Layers.size - 1].layer[i].activation
        CGradient[target] -= 1.0

        for i in range(self.Layers[self.Layers.size - 1].size):
            self.Layers[self.Layers.size - 1].zVector[i] = sigmoidDeriv(self.Layers[self.Layers.size - 1].zVector)

        return np.multiply(CGradient, self.endLayer.zVector)

    def cout(self):
        print("Start Layer:")
        self.Layers[0].coutNeurons()
        print()
        print("First Layer:")
        self.Layers[1].coutNeurons()
        print()
        print("Second Layer:")
        self.Layers[2].coutNeurons()
        print()
        print("End Layer:")
        self.Layers[3].coutNeurons()
        

In [148]:
# MNIST Dataset: 28 x 28 = 784
import pickle

#net = Network(784, 16, 16, 10)
net = Network(1, 1, 1, 1)

with open("dataset/pickled/first_10_data_train.pickle", "rb") as infile:
    data_batch = pickle.load(infile)
with open("dataset/pickled/first_10_label_train.pickle", "rb") as infile:
    label_batch = pickle.load(infile)

#with open("network.pickle", "rb") as infile:
#   net = pickle.load(infile)

#for i in range(0, len(data_batch)):
#    net.prepForFullForwardPropagation(data_batch[i])
#    net.fullForwardPropagation(label_batch[i])
#    print()

net.fullForwardPropagation(1)
net.cout()

with open("network.pickle", "wb") as outfile:
    pickle.dump(net, outfile)

Target is: 1
Cost is: 0.19269890976316992
Start Layer:
Neurons:
Activation:  0.4057328716006334
Bias:  0.8895598103710736
Incoming Weights:  []
Error:  0


First Layer:
Neurons:
Activation:  0.6365247276217039
Bias:  0.5427383300862858
Incoming Weights:  [0.0433134]
Error:  0


Second Layer:
Neurons:
Activation:  0.6918417090694724
Bias:  0.18670595488742314
Incoming Weights:  [0.97724035]
Error:  0


End Layer:
Neurons:
Activation:  0.6208041716405744
Bias:  0.2979621337912949
Incoming Weights:  [0.28185752]
Error:  0

